# Different Types of Semantic Functions

- *Inline*: inline functions are written in your chosen native language. You create the prompt, any variables, and configuration options in your desired language.
- *Semantic*: semantic functions are written in something closer to natural language. While this may initially seem too basic or limited, this non-traditional code can often be a very powerful approach in this A.I. world. ("English is the new programming language.")
- *Native*: like inline functions, native functions are written in your chosen native language, but they allow you to wrap any code logic as a function to be used within the kernel.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.15.230531.5-preview"

#!import ../config/SettingsHelper.cs

using Microsoft.SemanticKernel;

MySettings settings = Settings.LoadFromFile();

// Configure AI backend used by the kernel
var builder = new KernelBuilder();
if (settings.Type == "azure")
    builder.WithAzureTextCompletionService(settings.AzureOpenAI.CompletionsDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.WithOpenAITextCompletionService(settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
IKernel kernel = builder.Build();

## Inline Function

You can create a function alongside the rest of your code. These inline functions are very easy to implement and may allow for some very complex logic, but it potentially sacrifices some reusability and maintainability. 

In [ ]:
// define your inline function
string functionDefinition = """
Write the following words in reverse order: {{$input}}
""";
var reverseFunction = kernel.CreateSemanticFunction(functionDefinition);

// pass your prompt to your OpenAI instance and retrieve the completion
string prompt = "If you know the enemy and know yourself, you need not fear the result of a hundred battles. If you know yourself but not the enemy, for every victory gained you will also suffer a defeat. If you know neither the enemy nor yourself, you will succumb in every battle.";
var completion = await reverseFunction.InvokeAsync(prompt);

Console.WriteLine(completion);

# Semantic Functions

Semantic functions are especially interesting. Up to this point, we defined our functions using traditional C# code and there's a lot of great use cases for doing so. But in a world of A.I., why can't we just use a more natural language to define our function? A semantic function allows for exactly that!

Semantic functions are defined by two files - and a particular folder structure. The top-level folder is the plugin (formerly known as a skill) and it contains a variety of related functions. Each function is defined by two files: A config.json file defines the parameters to pass to OpenAI's API and the skprompt.txt file defines the actual prompt and any variables.

In [ ]:
using System.IO;

var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "Skills");

var mySkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "TranslateSkill");

string prompt = "Did you ever dance with the devil in the pale moonlight?";

var result = await kernel.RunAsync(prompt, mySkill["PigLatinSkill"]);

Console.WriteLine(result);

# Native Functions

Like inline functions, native functions are written in code. Since it's "just code", native functions provide the same opportunity to do anything you could do in code (basically everything! :) 

In [ ]:
#!import Skills/OldeEnglishSkill/OldeEnglishSkill.cs

var oeSkill = kernel.ImportSkill(new OldeEnglishSkill());

var context = kernel.RunAsync(
    "Death cannot stop true love. All it can do is delay it for a while.", 
    oeSkill["Translate"]);

Console.WriteLine(context.Result);

Hmmm...that may not have been the output we expected, but it is correct.. Unlike the semantic and inline functions we saw above, native functions aren't intrinsically wired into OpenAI. Native functions allow you to take any custom code and treat it as a skill/plugin that can be used within the kernel.

If you want to incorporate AI into your function and need to work in code, consider inline functions first. But what if you really want to add AI into your native function (or you really just want to see the Olde English interpretation of the quote above :) ).

In [ ]:
#!import Skills/OldeEnglishSkill/OldeEnglishSkill2.cs

var oeSkill = kernel.ImportSkill(new OldeEnglishSkill2());

var context = kernel.RunAsync(
    "Death cannot stop true love. All it can do is delay it for a while.", 
    oeSkill["Translate"]);

Console.WriteLine(context.Result);

## Exercise

With each of the above functions now associated with the kernel, write code in the box below to invoke each function with your own input prompt and write the output of each to the Console window.

# Exercise

Add your own semantic function to the Tutorial02 folder and test that you get the desired result.
Add your own native function to the Tutorial02 folder and test that you get the desired result.

You've achieved an important milestone because functions and skills/plugins are important building blocks of Semantic Kernel!

You can probably imagine all kinds of things you could start creating! But check out the next tutorial because it's important we start putting all of these building blocks together.